# Economic Indicators

This code file is meant to capture the relevant economic indicators for plotting and analysis.

Outstanding tasks include:
* Add data for more economic indicators from the remaining Fed districts
* Uncover how to set the major ticks 1 yr apart
* Clean up the code
* Perform time series lag correlation analysis

In [30]:
# !pip install openpyxl

In [31]:
import pandas as pd
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from datetime import datetime
from openpyxl import Workbook
import hvplot.pandas

In [70]:
# Read data
# New York Empire State data
empire_df = pd.read_csv('./Inputs/ESMS_SeasonallyAdjusted_Diffusion.csv')
empire_clean_df = empire_df[['surveyDate','GACDISA']]
empire_clean_df = empire_clean_df.rename(columns={
    "surveyDate":"Date",
    "GACDISA":"Empire_St"
})
empire_clean_df = empire_clean_df.set_index(['Date'])
empire_clean_df.index = pd.to_datetime(empire_clean_df.index)
empire_clean_df.tail()

,Empire_St
Date,
2021-04-30,26.3
2021-05-31,24.3
2021-06-30,17.4
2021-07-31,43.0
2021-08-31,18.3


In [66]:
# Read data
# Philadelphia Business Outlook Survey data
philly_df = pd.read_csv('./Inputs/bos_dif.csv')
philly_clean_df = philly_df[['DATE','GAC']]
philly_clean_df = philly_clean_df.rename(columns={
    "DATE":"Date",
    "GAC":"Philly"
})
# philly_clean_df[['Month','Year']] = philly_clean_df['Date'].str.split('-',expand=True)
# philly_clean_df['Year']='19'+philly_clean_df['Year'].astype(str)
# philly_clean_df['Year'] = philly_clean_df['Year'].apply(lambda x: '19'+philly_clean_df['Year'][x].astype(str) if x > 50,else '20'+philly_clean_df['Year'][x].astype(str))
# '19'+philly_clean_df['Year'].astype(str)
# philly_clean_df.loc[philly_clean_df['Year'] >= 50, 'Year']= '19' + philly_clean_df['Year']
# philly_clean_df.loc[philly_clean_df['Year'] < 50, 'Year']= '20' + philly_clean_df['Year']
philly_clean_df

,Date,Philly
0,May-68,32.2
1,Jun-68,18.4
2,Jul-68,33.3
3,Aug-68,-3.9
4,Sep-68,9.5
...,...,...
635,Apr-21,50.2
636,May-21,31.5
637,Jun-21,30.7
638,Jul-21,21.9


In [67]:
philly_clean_df = philly_clean_df.set_index(['Date'])
philly_clean_df

,Philly
Date,
May-68,32.2
Jun-68,18.4
Jul-68,33.3
Aug-68,-3.9
Sep-68,9.5
...,...
Apr-21,50.2
May-21,31.5
Jun-21,30.7


In [68]:
philly_clean_df.index = pd.to_datetime(philly_clean_df.index, infer_datetime_format=True,format="%b-%y")
philly_clean_df

,Philly
Date,
2068-05-01,32.2
2068-06-01,18.4
2068-07-01,33.3
2068-08-01,-3.9
2068-09-01,9.5
...,...
2021-04-01,50.2
2021-05-01,31.5
2021-06-01,30.7


In [89]:
# Read data
# Kansas City Manufacturing Survey data
kansas_df = pd.read_excel('./Inputs/ManufSurvey_Augu_PR_2021-8-26-Table2.xlsx', engine='openpyxl')
# philly_clean_df = philly_df[['DATE','GAC']]
# philly_clean_df = philly_clean_df.rename(columns={
#     "DATE":"Date",
#     "GAC":"Philly"
# })
# philly_clean_df = philly_clean_df.set_index(['Date'])
# # philly_clean_df.index = pd.to_datetime(philly_clean_df.index)
# philly_clean_df.tail()

kansas_df

,Table2,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 233,Unnamed: 234,Unnamed: 235,Unnamed: 236,Unnamed: 237,Unnamed: 238,Unnamed: 239,Unnamed: 240,Unnamed: 241,Unnamed: 242
0,Historical Manufacturing Survey Indexes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,2001-07-31 00:00:00,2001-08-31 00:00:00,2001-09-30 00:00:00,2001-10-31 00:00:00,2001-11-30 00:00:00,2001-12-31 00:00:00,2002-01-31 00:00:00,2002-02-28 00:00:00,2002-03-31 00:00:00,...,2020-11-30 00:00:00,2020-12-31 00:00:00,2021-01-31 00:00:00,2021-02-28 00:00:00,2021-03-31 00:00:00,2021-04-30 00:00:00,2021-05-31 00:00:00,2021-06-30 00:00:00,2021-07-31 00:00:00,2021-08-31 00:00:00
2,Versus a Month Ago,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,(seasonally adjusted),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Composite Index,-16,0,-11,-10,-9,-8,2,-3,6,...,11,14,17,24,26,31,26,27,30,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,Capital expenditures,0,-15,-19,-18,-24,-14,-8,-7,0,...,14,11,14,28,23,30,22,33,29,26
78,New orders for exports,13,8,-3,3,-1,0,2,3,13,...,7,11,10,12,5,9,18,16,17,5
79,Supplier delivery time,-5,-3,-3,4,-4,3,3,6,4,...,28,16,17,34,28,16,13,21,18,32
80,Inventories: Materials,-29,-15,-21,-28,-30,-8,-17,-3,4,...,4,9,4,11,24,17,18,24,10,24


In [90]:
kc_clean_df = kansas_df.drop([0,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,
                17,18,19,20,21,22,23,24,25,26,27,28,29,
                30,31,32,33,34,35,37,38,39,40,41,42,43,
                44,45,46,47,48,49,50,51,52,53,54,55,56,
                57,58,59,60,61,62,63,64,65,66,67,68,69,
                70,71,72,73,74,75,76,77,78,79,80,81])

In [91]:
kc_clean_df2 = kc_clean_df.T
kc_clean_df2 = kc_clean_df2.rename(columns={
    1:"Date",
    36:"Composite_Index"})
kc_clean_df2 = kc_clean_df2.drop(['Table2'])
kc_clean_df2 = kc_clean_df2.set_index(['Date'])
kc_clean_df2

,Composite_Index
Date,
2001-07-31,-23
2001-08-31,-12
2001-09-30,-16
2001-10-31,-18
2001-11-30,-24
...,...
2021-04-30,35
2021-05-31,43
2021-06-30,43


# New York Empire State Manufacturing Survey

In [75]:
empire_plot = empire_clean_df.hvplot.line(title='New York Empire State Mfg Survey',grid=True)
empire_plot

:Curve   [Date]   (Empire_St)

# Philadelphia Business Outlook Survey

In [47]:
philly_plot = philly_clean_df.hvplot.line(grid=True)
philly_plot

:Curve   [Date]   (Philly)

# Kansas City Manufacturing Survey

In [94]:
kc_plot = kc_clean_df2.hvplot.line(title='Kansas City Mfg Survey - Non Seasonally Adjusted YoY Composite Index',
                                   grid=True)
kc_plot

:Curve   [Date]   (Composite_Index)